# AI for BI

Generating a chart

We'll use CrewAI for the processing. This includes a Python interpreter which will be used to create the charts. You must have Docker installed locally to run the code automatically.

Running code generated by an LLM is potentially dangerous as there is n guarantee that the code wil be safe. Running the code in a Docker container, however, provides a sandbox for the code to run in and isolates it from the application code and local operating system.


Import the necessary libraries and set the LLM model (defaults to using OpenAI)

In [ ]:
from crewai import Agent, Task, Crew
llm = "gpt-4o-mini"

Import the tools required to read and write files

In [ ]:
from crewai_tools import tool
from crewai_tools import FileReadTool, FileWriterTool

# Initialize the tool to read any files the agents knows or lean the path for
file_read_tool = FileReadTool()
file_writer_tool = FileWriterTool()

Set up the ``chart_agent``

In [ ]:
# Define agent

chart_agent = Agent(
        role="Chart creator",
        goal="""Read the data provided and create a chart from that data.
                If you are given specific instructions on how to draw the chart then follow them,
                if not then create a chart that best represents all of the data""",
        backstory="You aim is to read and analyse sales data and create a chart",
        tools=[file_read_tool, file_writer_tool],
        allow_delegation=False,
        llm=llm,
        allow_code_execution=True
    )



The following defines a function that will create a task to create a markdown summary and chart for each data file and set up a crew.

We define the files and then loop through them calling the ``create_charts`` function to write the individual markdown files.


In [ ]:
files = [
    {
        'data_file_name':'sales_product_cat.csv',
        'chart_file_name': 'sales_product_summary.png',
        
    },
    {
        'data_file_name': 'monthly_sales.csv',
        'chart_file_name': 'monthly_sales.png',
    },
    {
        'data_file_name': 'sales_by_region.csv',
        'chart_file_name': 'sales_by_region.png',
    }
]



In [ ]:
create_chart = Task(
    description="""Create a chart for {data_file_name} and save it in {chart_file_name}.'
                    """,
    expected_output="""A matplotlib chart""",
    agent=chart_agent,
    tools=[file_read_tool, file_writer_tool]
)

# Define the crew
crew = Crew(
    agents=[chart_agent],
    tasks=[create_chart],
    verbose=True
)
result = crew.kickoff_for_each(inputs=files)

In [ ]:
data_analysis_agent = Agent(
        role="Data Analyser",
        goal="""You aim is to read and analyse sales data. You should
                then write a report on sales performance that includes an executive summary.
                """,
        backstory="You are assigned to perform sales analysis for a company",
        tools=[file_read_tool, file_writer_tool],
        allow_delegation=False,
        llm=llm,
        allow_code_execution=False
    )


In [13]:
write_report = Task(
    description=f"""The following contains a set of data files and corresponding charts:
                        {files}
                    Write report in Markdown that includes and overview of all of the sales data and incorporate 
                    the corresponding charts.

                    If the information is available, or you can calculate it,
                    try and answer the following questions: 
                    1. What has been the overall revenue for the period reported on?
                    2. What are the top selling 5 items and what revenue have they each generated?
                    3. In which regions have there been the most sales and what items are popular those regions?
                    4. What sort of growth has there been over the reporting period?
                    5. Are there any trends that you can detect?

                    The overview of the data and the corresponding charts from {files} should be included in an appendix.
                    
                    Save the result in the file './report.md'.
                    """,
    expected_output="""A markdown file""",
    agent=data_analysis_agent,
    tools=[file_read_tool, file_writer_tool]
)

# Define the crewwrite_overview
crew = Crew(
    agents=[data_analysis_agent],
    tasks=[write_report],
    verbose=True
)
result2 = crew.kickoff()

2024-11-13 20:11:09,086 - 13324 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Data Analyser
## Task: The following contains a set of data files and corresponding charts:
                        [{'data_file_name': 'sales_product_cat.csv', 'chart_file_name': 'sales_product_summary.png'}, {'data_file_name': 'monthly_sales.csv', 'chart_file_name': 'monthly_sales.png'}, {'data_file_name': 'sales_by_region.csv', 'chart_file_name': 'sales_by_region.png'}]
                    Write report in Markdown that includes and overview of all of the sales data and incorporate 
                    the corresponding charts.

                    If the information is available, or you can calculate it,
                    try and answer the following questions: 
                    1. What has been the overall revenue for the period reported on?
                    2. What are the top selling 5 items and what revenue have they each generated?
                    3. In which regions have there been the most sales and what items are popular those regions?
                  

![](image.png)